In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google'

In [ ]:
%cd / content/gdrive/MyDrive/research
! pip3 install -r requirements.txt

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/research/src')

In [ ]:
# Update path for custom module support in Google Colab
import sys
sys.path.append('/content/gdrive/MyDrive/research/src')

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.datasets import CIFAR10, FashionMNIST

#from tqdm.autonotebook import tqdm, trange

from utils.nets import *
from utils.model_tools import *
from utils.dataset_tools import split_training_data
from utils.feature_extractor import *
from utils.cosine_similarity import *
from utils.gen_dataset import *

In [2]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
model_dir = './models/'
log_dir = './logs/'
data_dir = './data/'
datasets_dir = './datasets/'

model_selection = 'linear' # linear | cnn | vgg
dataset_selection = 'fashionmnist' # cifar10 | fashionmnist

ckpt_file = model_dir + model_selection + '_' + dataset_selection + '_' + 'holdout_[8, 9].pt'
gen_dataset_path = datasets_dir + "g_" + dataset_selection + '/annotations'  + '.csv'
print(gen_dataset_path)

holdout_classes = [8, 9]
new_class = 8

batch_size = 32
num_classes = 9

./datasets/g_fashionmnist/annotations.csv


#### Hyperparameters

In [4]:
num_epochs = 10

initial_learning_rate = 0.001
final_learning_rate = 0.0001

# initial_lr * decay_rate^num_epochs = final_lr
decay_rate = (final_learning_rate/initial_learning_rate)**(1/num_epochs)

loss_fn = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=initial_learning_rate)
#lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

# Data Preparation

In [5]:
if dataset_selection == 'fashionmnist':
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),]) # Images are grayscale -> 1 channel
else:
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
if dataset_selection == 'cifar10':
    train_data = CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_data = CIFAR10(root='./data', train=False, download=True, transform=transform)
elif dataset_selection == 'fashionmnist':
    train_data = FashionMNIST(root='./data', train=True, download=True, transform=transform)
    test_data = FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [7]:
total_classes = len(np.unique(train_data.targets))

## FOL

In [8]:
if model_selection == 'linear':
    fol_model = add_output_nodes(ckpt_file, device, arch='linear')
    fol_model.input_layer.requires_grad_(False)
elif model_selection == 'cnn-demo':
    fol_model = add_output_nodes(ckpt_file, device, arch='cnn-demo')
    fol_model.conv1.requires_grad_(False)
    fol_model.conv2.requires_grad_(False)
    fol_model.fc1.requires_grad_(False)
elif model_selection == 'cnn':
    fol_model = add_output_nodes(ckpt_file, device, arch='cnn')
    fol_model.conv_block1.requires_grad_(False)
    fol_model.conv_block2.requires_grad_(False)
    fol_model.conv_block3.Conv5.requires_grad_(False)
    fol_model.conv_block3.Relu5.requires_grad_(False)
    fol_model.conv_block3.BN5.requires_grad_(False)
    
fol_model = fol_model.to(device)

input_size 784
num_outputs 9


In [9]:
fol_optimizer = torch.optim.Adam(fol_model.parameters(), lr=initial_learning_rate)
fol_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=fol_optimizer, gamma=decay_rate)

In [10]:
# training on fully interleaved data
included_data, excluded_data = split_training_data(train_data, [8])
train_fol_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)

# test on the full 9 classes (old classes + new one, still excluding one)
included_data, excluded_data = split_training_data(test_data, [8])
test_fol_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)

In [11]:
model_file_fol = model_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + '[8]' + 'fol' + '.pt'

recall_file_fol = log_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + '[8]' + 'fol' + '_recall.npy'
train_losses_file_fol = log_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + '[8]' + 'fol' + '_train_loss.txt'
test_losses_file_fol = log_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + '[8]' + 'fol' + '_test_loss.txt'

### Training Loop

In [12]:
train_losses = []
test_losses = []
#t = trange(num_epochs)
t = range(num_epochs)
y_preds = []
y_actuals = []

for epoch in range(3):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = train(train_fol_loader, fol_model, loss_fn, fol_optimizer, device, swap=True, swap_labels=[9,8])
    test_loss = test(test_fol_loader, fol_model, loss_fn, device, swap=True, swap_labels=[9,8])
    #print('y_pred:', y_pred[:2])
    #print('y_actual:', y_actual[:2])
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    #y_preds.append(y_pred)
    #y_actuals.append(y_actual)
    
    fol_lr_scheduler.step()
    
torch.save(fol_model.state_dict(), model_file_fol)

#recalls = get_recall_per_epoch(y_actuals, y_preds, num_classes)
#np.save(recall_file_fol, recalls)

#with open(train_losses_file_fol, 'w') as fp:
#    for s in train_losses:
#        fp.write("%s\n" % s)
        
#with open(test_losses_file_fol, 'w') as fp:
#    for x in test_losses:
#        fp.write("%s\n" % x)

print("Done!")

# Ok I'm struggling with getting data loading correct for FOL but I'm not sure we even need it

Epoch 1
-------------------------------
loss: 15.908536  [    0/54000]
loss: 0.399133  [32000/54000]
Test Error: 
 Accuracy: 81.3%, Avg loss: 0.560368 

Epoch 2
-------------------------------
loss: 0.441855  [    0/54000]
loss: 0.465772  [32000/54000]
Test Error: 
 Accuracy: 81.9%, Avg loss: 0.518535 

Epoch 3
-------------------------------
loss: 0.429902  [    0/54000]
loss: 0.721021  [32000/54000]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.506766 

Done!


## SWIL

In [37]:
if model_selection == 'linear':
    swil_model = add_output_nodes(ckpt_file, device, arch='linear')
    swil_model.input_layer.requires_grad_(False)
elif model_selection == 'cnn-demo':
    swil_model = add_output_nodes(ckpt_file, device, arch='cnn-demo')
    swil_model.conv1.requires_grad_(False)
    swil_model.conv2.requires_grad_(False)
    swil_model.fc1.requires_grad_(False)
    
swil_model = swil_model.to(device)

input_size 784
num_outputs 9


In [38]:
swil_optimizer = torch.optim.Adam(swil_model.parameters(), lr=initial_learning_rate)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=swil_optimizer, gamma=decay_rate)

In [39]:
fmnist_classes = list(range(8)) + [9]

# might not need these
FMNIST_trainloader_gen = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
FMNIST_testloader_gen = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

class_subsets, class_idxs, subset_size = generate_dls(train_data, fmnist_classes)

In [40]:
with open(r'./data/fmnist_sim_scores_boot.txt', 'r') as fp:
    sim_scores = [float(i) for i in fp.readlines()]

sim_sum = sum(sim_scores)

sim_norms = [x/sim_sum for x in sim_scores]

boots_sample_size = 75
sim_sample_sizes = [27 if x < 0.2 else int(x * boots_sample_size*3.52) for x in sim_norms] + [75]

In [41]:
sim_sample_sizes = [i*7 for i in sim_sample_sizes]
print(sim_sample_sizes)

[189, 189, 189, 189, 189, 385, 189, 399, 525]


In [42]:
from random import sample

sampled_idxs = []

for i in range(len(fmnist_classes)):
    idx_sample = sample(class_idxs[i].tolist(), sim_sample_sizes[i])
    sampled_idxs += idx_sample

swil_train_subset = torch.utils.data.Subset(train_data, sampled_idxs)

swil_train_dl = torch.utils.data.DataLoader(swil_train_subset, batch_size=1, shuffle=True, num_workers=2)

included_data, excluded_data = split_training_data(test_data, [8])
test_swil_loader = DataLoader(included_data, batch_size=batch_size, shuffle=True, num_workers=2)

### Training Loop

In [43]:
model_file = model_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + '[8]' + '_swil.pt'
train_losses_file = model_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + '[8]' + '_swil_train_loss.txt'
test_losses_file = model_dir + model_selection + '_' + dataset_selection + '_' + 'holdout' + '_' + '[8]' + 'swil_test_loss.txt'
train_losses = []
test_losses = []
#t = trange(num_epochs)
t = range(num_epochs)
#y_preds = []
#y_actuals = []

for epoch in t:
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = train(swil_train_dl, swil_model, loss_fn, swil_optimizer, device, swap=True, swap_labels=[9,8])
    test_loss = test(test_swil_loader, swil_model, loss_fn, device, swap=True, swap_labels=[9,8])
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    #y_preds.append(y_pred)
    #y_actuals.append(y_actual)
    
    #t.set_description(f"Epoch {epoch} train loss: {epoch_loss_train[-1]:.3f}")
    lr_scheduler.step()
    
torch.save(swil_model.state_dict(), model_file)

#recalls = get_recall_per_epoch(y_actuals, y_preds, 9)
#np.save(recall_file, recalls)

with open(train_losses_file, 'w') as fp:
    for s in train_losses:
        fp.write("%s\n" % s)
        
with open(test_losses_file, 'w') as fp:
    for x in test_losses:
        fp.write("%s\n" % x)

print("Done!")

Epoch 1
-------------------------------
loss: 16.739058  [    0/ 2443]
loss: 0.083764  [ 1000/ 2443]
loss: 0.173965  [ 2000/ 2443]
Test Error: 
 Accuracy: 76.1%, Avg loss: 0.792950 

Epoch 2
-------------------------------
loss: 2.304740  [    0/ 2443]
loss: 0.000275  [ 1000/ 2443]
loss: 0.285808  [ 2000/ 2443]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.609608 

Epoch 3
-------------------------------
loss: 0.000000  [    0/ 2443]
loss: 2.611470  [ 1000/ 2443]
loss: 0.000020  [ 2000/ 2443]
Test Error: 
 Accuracy: 78.7%, Avg loss: 0.649275 

Epoch 4
-------------------------------
loss: 1.564470  [    0/ 2443]
loss: 0.280462  [ 1000/ 2443]
loss: 0.687943  [ 2000/ 2443]
Test Error: 
 Accuracy: 80.8%, Avg loss: 0.574304 

Epoch 5
-------------------------------
loss: 0.159231  [    0/ 2443]
loss: 1.718632  [ 1000/ 2443]
loss: 0.000000  [ 2000/ 2443]
Test Error: 
 Accuracy: 81.2%, Avg loss: 0.557976 

Epoch 6
-------------------------------
loss: 1.463626  [    0/ 2443]
loss: 0.012542  [ 1

TypeError: not all arguments converted during string formatting